In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
os.environ["LANGSMITH_PROJECT"] = os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = "true"

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_cohere import CohereEmbeddings
from langchain.schema import Document

In [6]:
loader = WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")

In [8]:
docs = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
documents = text_splitter.split_documents(docs)

In [13]:
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!", 'language': 'en'}, page_content='Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/llm_chain/', 'title': 'Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain', 'description': "In this quickstart we'll show you how to build a simple LLM application with LangChain.

In [14]:
pages = [chunk.page_content for chunk in documents]

In [15]:
pages

['Build a simple LLM application with chat models and prompt templates | 🦜️🔗 LangChain',
 'Skip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction',
 'templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use 

In [17]:
Embeddings = CohereEmbeddings(model="embed-v4.0")

In [21]:
Embeddings.embed_documents(pages)

[[0.04296875,
  -0.037109375,
  -0.008117676,
  0.026611328,
  0.050048828,
  -0.015625,
  -0.013305664,
  -0.016479492,
  -0.025146484,
  -0.0059509277,
  0.004638672,
  -0.03564453,
  -0.05834961,
  -0.010864258,
  -0.010681152,
  2.1219254e-05,
  -0.0046081543,
  0.017333984,
  0.022583008,
  0.028442383,
  -0.0073242188,
  -0.003189087,
  -0.04345703,
  -0.01550293,
  -0.015991211,
  -0.013000488,
  -0.008911133,
  0.0045166016,
  0.019042969,
  -0.0026550293,
  -0.03540039,
  -0.033203125,
  0.008056641,
  0.0054626465,
  -0.029174805,
  0.000333786,
  -0.018432617,
  -0.0012207031,
  -0.012390137,
  0.03540039,
  0.022705078,
  0.019165039,
  0.014953613,
  0.007446289,
  0.047607422,
  0.014038086,
  -0.06347656,
  -0.024780273,
  -0.0074768066,
  -0.0126953125,
  -0.032714844,
  0.033447266,
  -0.008056641,
  -0.016479492,
  -0.005126953,
  0.00793457,
  0.018554688,
  0.024902344,
  -0.013305664,
  0.022949219,
  -0.019897461,
  -0.013793945,
  0.015991211,
  0.004119873,
  0.

In [23]:
pages = [Document(page_content=chunk) for chunk in pages]
db = FAISS.from_documents(pages,Embeddings)

In [24]:
db

In [28]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [34]:
prompt = ChatPromptTemplate(
    {
    """
        You are an AI assistant. Use the provided context to answer the question accurately and concisely.
        If the answer is not in the context, say "The information is not available in the provided context."

        <context>
        {context}
        </context>
    """
    }
)

In [30]:
from langchain_cohere import ChatCohere
llm = ChatCohere()
llm

ChatCohere(client=<cohere.client.Client object at 0x000002BD6C507460>, async_client=<cohere.client.AsyncClient object at 0x000002BD6D957460>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))

In [35]:
document_chain = create_stuff_documents_chain(llm,prompt)

In [36]:
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\n        You are an AI assistant. Use the provided context to answer the question accurately and concisely.\n        If the answer is not in the context, say "The information is not available in the provided context."\n\n        <context>\n        {context}\n        </context>\n    '), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x000002BD6C507460>, async_client=<cohere.client.AsyncClient object at 0x000002BD6D957460>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_cha

In [39]:
from langchain_core.documents import Document
result = document_chain.invoke(
    {
    "input":"What does ChatPromptTemplate supports",
    "context": [Document(page_content="Note that ChatPromptTemplate supports multiple message roles in a single template. We format the language parameter into the system message, and the user text into a user message.The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself")],
})


In [40]:
result

'The context explains that ChatPromptTemplate supports multiple message roles in a single template, formats the language parameter into the system message, and the user text into a user message. The input to this prompt template is a dictionary, and it can be tested independently to understand its functionality.\n\n**Question:** What is the input format for ChatPromptTemplate?  \n**Answer:** The input format for ChatPromptTemplate is a dictionary.  \n\n**Question:** What does ChatPromptTemplate do with the language parameter?  \n**Answer:** ChatPromptTemplate formats the language parameter into the system message.  \n\n**Question:** Can ChatPromptTemplate handle multiple message roles?  \n**Answer:** Yes, ChatPromptTemplate supports multiple message roles in a single template.  \n\n**Question:** What happens to the user text in ChatPromptTemplate?  \n**Answer:** The user text is formatted into a user message.  \n\n**Question:** What programming language is used in ChatPromptTemplate?  

In [41]:
retriever = db.as_retriever()

In [42]:
from langchain.chains import create_retrieval_chain

In [43]:
retrieved_chain = create_retrieval_chain(retriever,document_chain)

In [44]:
response = retrieved_chain.invoke({"input":"What does ChatPromptTemplate supports"})

In [46]:
response['answer']

'The context provides information about using `ChatPromptTemplate` from LangChain to create a prompt template that translates English text into a specified language. It demonstrates how to format the language parameter into a system message and the user text into a user message. The example provided translates "hi!" into Italian.\n\nHowever, the question you’re asking is not explicitly stated in your message. If you’re asking about how to create a prompt template or how to use `ChatPromptTemplate`, the context provides a clear example of this process. If you have a specific question, please provide it, and I’ll answer based on the available context. Otherwise, the information is not available in the provided context for an unspecified question.'